In [ ]:
# %%capture
%pip install "unstructured[all-docs]"

In [2]:
# # Warning control
# import warnings
# warnings.filterwarnings('ignore')

In [3]:
from IPython.display import JSON

import json

from unstructured.partition.html import partition_html
from unstructured.partition.pdf import partition_pdf
from unstructured.partition.image import partition_image
from unstructured.staging.base import dict_to_elements, elements_to_json

c:\Users\Hemant.Singhsidar\Desktop\Unstructured\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# filename = "CaseStudies.pdf"

In [4]:
poppler_path = r"C:/Users/Hemant.Singhsidar/Downloads/Release-24.08.0-0/poppler-24.08.0/Library/bin"

In [6]:
# from unstructured.partition.pdf import partition_pdf

# elements = partition_pdf(filename=filename,
#                          infer_table_structure=True,
#                          strategy='hi_res',
#            )

In [5]:
filename = "PDF Pages/2111.15664v5-Pages/page_1.png"

In [6]:
from unstructured.partition.image import partition_image

elements = partition_image(filename=filename,
                         infer_table_structure=True,
                         strategy='hi_res',
           )

In [7]:
len(elements)

12

In [8]:
elements

In [9]:
import json

element_dict = [el.to_dict() for el in elements]
output = json.dumps(element_dict, indent=2)
print(output)

unique_types = set()

for item in element_dict:
    unique_types.add(item['type'])

print(unique_types)

[
  {
    "type": "Title",
    "element_id": "194bb9484908dab35dc6eb221f16ea64",
    "text": "OCR-free Document Understanding Transformer",
    "metadata": {
      "detection_class_prob": 0.5052953958511353,
      "coordinates": {
        "points": [
          [
            91.509521484375,
            89.04460144042969
          ],
          [
            91.509521484375,
            130.8874969482422
          ],
          [
            1053.1043701171875,
            130.8874969482422
          ],
          [
            1053.1043701171875,
            89.04460144042969
          ]
        ],
        "system": "PixelSpace",
        "layout_width": 1150,
        "layout_height": 1709
      },
      "last_modified": "2025-02-10T13:44:24",
      "filetype": "PNG",
      "languages": [
        "eng"
      ],
      "page_number": 1,
      "file_directory": "PDF Pages/2111.15664v5-Pages",
      "filename": "page_1.png"
    }
  },
  {
    "type": "UncategorizedText",
    "element_id": "8ce

In [11]:
import json

# Convert elements to dictionary format
elements_dict = [el.to_dict() for el in elements]  # Assuming `elements` is a list of objects

# Target types to extract
target_types = {"Table", "Image"}

# Dictionary to store bounding boxes
bounding_boxes = {}

# Counter for total extracted boxes
total_boxes = 0

for item in elements_dict:
    if item["type"] in target_types:
        # Extract bounding box coordinates
        points = item["metadata"]["coordinates"]["points"]
        
        # Calculate bounding box as [x_min, y_min, x_max, y_max]
        x_min = min(p[0] for p in points)
        y_min = min(p[1] for p in points)
        x_max = max(p[0] for p in points)
        y_max = max(p[1] for p in points)
        
        # Store bounding box using element_id for reference
        bounding_boxes[item["element_id"]] = {
            "type": item["type"],
            "bbox": [x_min, y_min, x_max, y_max],
            "page_number": item["metadata"]["page_number"],
            "filename": item["metadata"]["filename"]
        }
        
        total_boxes += 1

# Print total count of bounding boxes
print(f"Total bounding boxes extracted: {total_boxes}")

# Print bounding boxes data
print(json.dumps(bounding_boxes, indent=2))

Total bounding boxes extracted: 1
{
  "34d555696d48156eac62d2a4600ae701": {
    "type": "Table",
    "bbox": [
      224.56277465820312,
      319.9187927246094,
      925.2555541992188,
      382.0470886230469
    ],
    "page_number": 1,
    "filename": "page_1.png"
  }
}


In [12]:
bounding_boxes

{'34d555696d48156eac62d2a4600ae701': {'type': 'Table',
  'bbox': [np.float64(224.56277465820312),
   np.float64(319.9187927246094),
   np.float64(925.2555541992188),
   np.float64(382.0470886230469)],
  'page_number': 1,
  'filename': 'page_1.png'}}

In [ ]:
import os
from pdf2image import convert_from_path
from PIL import Image

def extract_and_save_tables(pdf_path, bounding_boxes):
    """
    Extract tables from a PDF using bounding boxes and save them as images.

    Args:
        pdf_path (str): Path to the input PDF file.
        bounding_boxes (dict): Dictionary containing bounding box information.
    """
    # Convert PDF to images (one per page)
    images = convert_from_path(pdf_path, dpi=300)

    for table_id, data in bounding_boxes.items():
        page_num = data["page_number"] - 1  # Adjust for zero-based index
        filename = os.path.splitext(os.path.basename(data["filename"]))[0]
        bbox = data["bbox"]
        element_type = data["type"]

        # Define save directory
        save_dir = f"{filename}-{element_type}"
        os.makedirs(save_dir, exist_ok=True)

        # Open the corresponding page image
        page_img = images[page_num]

        # Crop the table region
        cropped_img = page_img.crop((bbox[0], bbox[1], bbox[2], bbox[3]))

        # Save the cropped image
        save_path = os.path.join(save_dir, f"{table_id}_{data['page_number']}.png")
        cropped_img.save(save_path, "PNG")

        print(f"Saved: {save_path}")

# Example Usage
pdf_path = "CaseStudies.pdf"  # Path to your PDF file

extract_and_save_tables(pdf_path, bounding_boxes)

In [ ]:
import os
import json
from pdf2image import convert_from_path
from PIL import Image, ImageDraw

def annotate_tables(pdf_path, output_data):
    """
    Extracts and annotates tables in a PDF document using bounding boxes from extracted data.

    Args:
        pdf_path (str): Path to the input PDF file.
        output_data (list): List of dictionaries containing element details.
    """
    # Convert PDF to images
    images = convert_from_path(pdf_path, dpi=300)

    # Process each detected element
    for element in output_data:
        if element["type"] == "Table":  # Only process tables
            page_num = element["metadata"]["page_number"] - 1  # Convert to zero-based index
            bbox_points = element["metadata"]["coordinates"]["points"]
            filename = os.path.splitext(os.path.basename(element["metadata"]["filename"]))[0]

            # Define save directory
            save_dir = f"{filename}-AnnotatedTables"
            os.makedirs(save_dir, exist_ok=True)

            # Open corresponding page image
            page_img = images[page_num].convert("RGB")

            # Draw bounding box annotation
            draw = ImageDraw.Draw(page_img)
            x_min, y_min = bbox_points[0]
            x_max, y_max = bbox_points[2]
            draw.rectangle([x_min, y_min, x_max, y_max], outline="red", width=3)

            # Save annotated image
            save_path = os.path.join(save_dir, f"{element['element_id']}_{page_num + 1}_annotated.png")
            page_img.save(save_path, "PNG")

            print(f"Saved annotated image: {save_path}")

# Example Usage
pdf_path = "CaseStudies.pdf"  # Path to the PDF file
output_data = json.loads(output)
# annotate_tables(pdf_path, output_data)

In [ ]:
output_data

In [ ]:
for element in output_data:
        if element["type"] == "Table":  # Only process tables
            page_num = element["metadata"]["page_number"] - 1  # Convert to zero-based index
            bbox_points = element["metadata"]["coordinates"]["points"]
            filename = os.path.splitext(os.path.basename(element["metadata"]["filename"]))[0]
            x_min, y_min = bbox_points[0]
            x_max, y_max = bbox_points[2]
            print(bbox_points)
            print(page_num)
            print(filename)
            print(x_min, y_min, x_max, y_max)


In [ ]:
import os
import cv2
import json
from pdf2image import convert_from_path
import numpy as np

# Load JSON if it's stored as a string
output_json_path = "output.json"  # Replace with your actual file path

# Check if the output is a JSON file, and load it properly
if os.path.exists(output_json_path):
    with open(output_json_path, "r", encoding="utf-8") as file:
        output = json.load(file)  # Ensure it's a dictionary or list
elif isinstance(output, str):  # If it's a string, parse it as JSON
    output = json.loads(output)

def annotate_tables(pdf_path, output_data):
    """
    Annotates detected tables on PDF pages with bounding boxes instead of cropping.

    Args:
        pdf_path (str): Path to the input PDF file.
        output_data (list): List of dictionaries containing extracted element details.
    """
    if not isinstance(output_data, list):  # Ensure it's a list
        raise ValueError("Expected 'output_data' to be a list of dictionaries.")

    # Convert PDF to images
    images = convert_from_path(pdf_path, dpi=300)
    print(f"✅ PDF converted to {len(images)} images.")

    if not images:
        print("❌ Error: No images extracted from the PDF!")
        return

    # Create a directory for annotated pages
    pdf_name = os.path.splitext(os.path.basename(pdf_path))[0]
    save_dir = f"{pdf_name}-AnnotatedPages"
    os.makedirs(save_dir, exist_ok=True)

    # Process each detected table and annotate
    for element in output_data:
        if isinstance(element, dict) and element.get("type") == "Table":  # Ensure it's a valid table entry
            try:
                page_num = element["metadata"]["page_number"] - 1  # Convert to zero-based index
                bbox_points = element["metadata"]["coordinates"]["points"]

                # Extract bounding box coordinates
                x_min, y_min = map(int, bbox_points[0])  # Top-left
                x_max, y_max = map(int, bbox_points[2])  # Bottom-right

                if page_num < 0 or page_num >= len(images):
                    print(f"❌ Error: Page number {page_num + 1} out of range.")
                    continue

                # Convert PIL image to OpenCV format properly
                page_img = np.array(images[page_num])  # Convert PIL to NumPy array
                page_img = cv2.cvtColor(page_img, cv2.COLOR_RGB2BGR)  # Convert RGB to BGR for OpenCV

                # Verify if the image is loaded correctly
                if page_img is None or page_img.size == 0:
                    print(f"❌ Error: Page {page_num + 1} could not be read properly.")
                    continue

                # Draw bounding box around the table
                color = (0, 0, 255)  # Red color for annotation
                thickness = 3  # Thickness of the bounding box
                cv2.rectangle(page_img, (x_min, y_min), (x_max, y_max), color, thickness)

                # Put a label near the table
                label_text = f"Table {element['element_id']}"
                cv2.putText(
                    page_img, label_text, (x_min, y_min - 10), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2
                )

                # Save the annotated image
                save_path = os.path.join(save_dir, f"annotated_page_{page_num + 1}.png")
                cv2.imwrite(save_path, page_img)

                print(f"✅ Annotated table on page {page_num + 1}, saved to: {save_path}")

            except Exception as e:
                print(f"❌ Error processing table {element['element_id']}: {e}")

# Example Usage
pdf_path = "CaseStudies.pdf"  # Path to the PDF file
annotate_tables(pdf_path, output)

# Crop tables and Images

In [ ]:
import cv2
import os

def extract_blocks(layout, image_path, output_dir):

  # Create the output directory if it doesn't exist
  if not os.path.exists(output_dir):
    os.makedirs(output_dir)

  im = cv2.imread(image_path)
  block_count = 0

  for l in layout:
    if l.type == 'Table' or 'Text':
      x_1 = round(l.block.x_1)
      y_1 = round(l.block.y_1)
      x_2 = round(l.block.x_2)
      y_2 = round(l.block.y_2)

      block_count += 1
      output_filename = f"{output_dir}/block_{block_count}.png"
      cv2.imwrite(output_filename, im[y_1:y_2, x_1:x_2])
      print(f"Block {block_count} saved to {output_filename}")  # Print the message

# Example usage:
img_path = "OUTPUTS\output_images\page_2.png"
extract_blocks(layout, img_path, 'TESTING\Extracted Tables Pub')

In [ ]:
tables = [el for el in elements if el.category == "Table"]

print(tables[0].text)
print(tables[0].metadata.text_as_html)

In [ ]:
tables

In [ ]:
len(tables)

In [ ]:
tables[5].text

In [ ]:
tables[0].metadata

In [ ]:
table_html = tables[3].metadata.text_as_html

In [ ]:
import pandas as pd

# Convert HTML table to pandas DataFrame
dfs = pd.read_html(table_html)
dfs

In [ ]:

# Assuming there's only one table, get the DataFrame
df = dfs[0]

# Now you have the DataFrame
print(df)


In [ ]:
df.shape

In [ ]:
df.head()